In [3]:
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType, IntegerType

In [4]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/19 16:07:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
census_df = spark.read.csv('data/NewYork.csv', header = True)
census_df.show(5)

+---+--------+-------+------+------+-----------+--------------------+--------------+--------+--------+----+----+----+----+--------+--------+-------+-------+-------+-------+--------+--------+---------+---------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+-------+-------+--------+--------+-------+-------+-------+-------+--------+--------+---------+---------+--------+--------+--------+--------+---------+---------+----------+----------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+--------+--------+---------+---------+--------+--------+--------+--------+---------+---------+----------+---------+---------+----------+-----------+----------+----------+---------+---------+----------+----------+----------+----------+----------+----------+----------+----------+---------+----------+---------+---------+----------+----------+----------+----------+----------+--------+-----

In [7]:
# Prefixes of the column names to drop
prefixes_to_drop = ['M', 'EP', 'SP', 'F']
# Identify columns to drop based on prefixes
columns_to_drop = [column for column in census_df.columns if any(column.startswith(prefix) for prefix in prefixes_to_drop)]
# Drop the columns identified by prefixes
census_df = census_df.drop(*columns_to_drop)

# Additional columns to drop by specific names
cols_to_drop = ['RPL_THEME1', 'RPL_THEME2', 'RPL_THEME3', 'RPL_THEME4', 'E_DAYPOP',
                'E_AFAM', 'E_HISP', 'E_ASIAN', 'E_AIAN', 'E_NHPI','E_TWOMORE', 
                'E_OTHERRACE', 'ST', 'STATE', 'ST_ABBR', 'STCNTY', 'LOCATION']
# Drop the specifically named columns
census_df = census_df.drop(*cols_to_drop)

#renaming a column
census_df = census_df.withColumnRenamed('RPL_THEMES', 'SVI_Rank')  # Ensure the original column name is 'RPL_THEMES'

# Show the first 5 rows of the DataFrame to verify changes
census_df.show(5)



+------+--------------+--------+----+----+--------+-------+-------+--------+---------+-------+-------+--------+--------+--------+--------+-------+--------+-------+-------+--------+--------+-------+
|COUNTY|     AREA_SQMI|E_TOTPOP|E_HU|E_HH|E_POV150|E_UNEMP|E_HBURD|E_NOHSDP|E_UNINSUR|E_AGE65|E_AGE17|E_DISABL|E_SNGPNT|E_LIMENG|E_MINRTY|E_MUNIT|E_MOBILE|E_CROWD|E_NOVEH|E_GROUPQ|SVI_Rank|E_NOINT|
+------+--------------+--------+----+----+--------+-------+-------+--------+---------+-------+-------+--------+--------+--------+--------+-------+--------+-------+-------+--------+--------+-------+
|Albany|0.914079496512|    2029| 909| 769|     687|    167|    349|     251|      177|    280|    565|     231|     149|      86|    1605|     14|       0|     23|     93|      13|  0.8532|    379|
|Albany|0.237787480434|    3263|1861|1382|    1331|     91|    594|     239|      121|    204|    623|     475|     336|      80|    2646|    205|       0|      0|    440|      71|   0.664|    278|
|Albany| 0